In [38]:
import pandas as pd
df = pd.read_csv('gossipcop_llm_rationales.csv')
rationale_names = {'td','ia','cs','itc'}
df[:10]

,content,label,time,source_id,split,td,td_pred,td_acc,ia,ia_pred,ia_acc,cs,cs_pred,cs_acc,itc,itc_pred,itc_acc
0,Congratulations might be in order for Miley Cy...,0,2018-06-22 14:58:20,gossipcop-2493749932,NaN,The source of the information is not reputable...,0,1,The image appears to be a candid photograph of...,1,0,The news text and the image of Miley Cyrus and...,1,0,The image shows Miley Cyrus and Liam Hemsworth...,1,0
1,Thousands are taking the streets to protest Pr...,0,2017-04-15 21:25:01,gossipcop-941805037,NaN,the news article is based on a real event that...,1,0,The image appears to be a photograph taken at ...,1,0,The text and image both describe a Tax March p...,1,0,The image shows a protest with a sign comparin...,1,0
2,We'd venture to say that Cindy Crawford's daug...,0,2016-03-18 20:27:55,gossipcop-2547891536,NaN,no suspicious information or language use in t...,1,0,The image appears to be a natural photograph w...,1,0,The news text and the image of Kaia Gerber wit...,1,0,"The image shows a person with blonde hair, whi...",1,0
3,"During the summer of 1995, John F. Kennedy Jr....",0,2017-06-29 20:40:00,gossipcop-5189580095,NaN,The text does not provide any credible sources...,0,1,The basis for judging the authenticity of the ...,1,0,The text provides a detailed account of a meet...,1,0,The image shows John F. Kennedy Jr. and Prince...,1,0
4,"Many stars saw great success in 2016, but the ...",0,2016-12-15 16:00:00,gossipcop-9588339534,NaN,the article provides factual information about...,1,0,The image contains a collage of unrelated cele...,0,1,The text and image are consistent with the new...,1,0,The image contains pictures of celebrities men...,1,0
5,Caitlyn Jenner is trying to keep her romantic ...,0,2018-08-07 17:46:56,gossipcop-8753274298,NaN,"the news is based on a real person and event, ...",1,0,The image appears to be a genuine photograph o...,1,0,The text and the image are consistent with eac...,1,0,The image shows two individuals who appear to ...,1,0
6,So while the internet is busy obsessing over J...,0,2018-07-09 09:01:04,gossipcop-9878194459,NaN,The article provides credible sources and quot...,1,0,The image appears to be a standard photograph ...,1,0,The news text and the image of Selena Gomez ar...,1,0,"The image shows Selena Gomez, who is mentioned...",1,0
7,Next stop on the 2018 award season train? The ...,0,2018-01-12 03:00:00,gossipcop-9521617242,NaN,the news text is a news summary from a reputab...,1,0,The image appears to be a still from a movie o...,1,0,The text is a news summary about the 23rd Crit...,1,0,The image shows a woman with a serious express...,1,0
8,"Angelina Jolie, 43, “really hasn’t come to ter...",0,2018-08-12 16:00:00,gossipcop-2210288941,NaN,The news article is based on a source close to...,1,0,The image appears to be a genuine photograph o...,1,0,The text provides a detailed and specific news...,1,0,"The image shows Angelina Jolie and Brad Pitt, ...",1,0
9,We should not be content to pass health care l...,0,2017-09-21 16:00:00,gossipcop-3783917182,NaN,the text presents a clear argument and is not ...,1,0,The image appears to be a genuine photograph o...,1,0,"The text is a news summary, and the image is a...",1,0,The image shows a man speaking into a micropho...,1,0


In [39]:
def filter_data(df):
    for r_name in rationale_names:
        df = df[ df[f'{r_name}_pred'] != -1 ]
    return df
        
print(f"before filtering : {df.shape[0]}")
df = filter_data(df)
print(f"after filtering : {df.shape[0]}")

for r_name in rationale_names:
    print(f"{r_name}: {(df[r_name] == None).sum()}")

before filtering : 8473
after filtering : 8157
td: 0
ia: 0
itc: 0
cs: 0


In [40]:


for r_name in rationale_names:
    acc = r_name+ "_acc"
    print(f"{acc}: {(df[acc]==1).sum()/ df.shape[0]}")
    

td_acc: 0.7490498957950227
ia_acc: 0.6981733480446243
itc_acc: 0.7125168566875076
cs_acc: 0.73421601078828


In [41]:
def split_data0(df,train_ratio=0.8,valid_ratio=0.1):
    train_nums = int(df.shape[0]*train_ratio)
    valid_nums = int(df.shape[0]*valid_ratio)
    return df.iloc[:train_nums],df.iloc[train_nums:train_nums+valid_nums],df.iloc[train_nums+valid_nums:]

def split_data(df,train_ratio=0.8,valid_ratio=0.1):
    real_data = df[df['label'] == 1]
    fake_data = df[df['label'] == 0]
    print(f"real_data count : {real_data.shape[0]}")
    print(f"fake_data count : {fake_data.shape[0]}")
    train_real,valid_real,test_real = split_data0(real_data,train_ratio,valid_ratio)
    train_fake,valid_fake,test_fake = split_data0(fake_data,train_ratio,valid_ratio)
    train_df,valid_df,test_df = pd.concat([train_real,train_fake]),pd.concat([valid_real,valid_fake]),pd.concat([test_real,test_fake])
    train_df['split'] = 'train'
    valid_df['split'] = 'valid'
    test_df['split'] = 'test'
    return train_df,valid_df,test_df


train_df,valid_df,test_df = split_data(df,train_ratio=0.8,valid_ratio=0.1)


train_df.shape[0],valid_df.shape[0],test_df.shape[0]

real_data count : 5988
fake_data count : 2169


(6525, 814, 818)

In [42]:
train_df.to_csv('train.csv',index=False)
valid_df.to_csv('valid.csv',index=False)
test_df.to_csv('test.csv',index=False)